In [ ]:
--select distinct Designation from ZydusMTManagement.ISPAttendanceMasterExport where month =1 and year=2026



WITH Attendance_CTE AS (
        SELECT
            
            M.EmpPositionCode,
            M.EmpPositionCodeStatus,
            A.UserName,
            A.FullName,
            A.Region,
            A.City,
            A.Designation,
            A.Status,
            A.[Date] AS Date2,
            M.Status AS chek,
            M.LastWorkingDate,
            CASE
                WHEN A.Status = 'Present' THEN 'P'
                WHEN A.Status = 'Absent' THEN 'A'
                WHEN A.Status = 'Leave' THEN 'L'
                WHEN A.Status = 'Compensatory Off' THEN 'CO'
                WHEN A.Status IN ('Week Off', 'WeekOff') THEN 'WO'
                WHEN A.Status = 'Meeting' THEN 'M'
                WHEN A.Status = 'Training' THEN 'T'
                WHEN A.Status IN ('National Hoilday', 'Holiday') THEN 'H'
                ELSE 'NA'
            END AS AttStatus,
            FORMAT(TRY_CONVERT(date, A.[Date], 105), 'dd') AS DayNum
        FROM PowerBI_LISV2.ZydusMTManagement.ISPAttendanceMasterExport AS A
        LEFT JOIN (
            SELECT K.*
            FROM (
                SELECT *,
                       RANK() OVER (PARTITION BY UserName ORDER BY UserMasterId DESC) AS RNK
                FROM PowerBI_LISV2.ZydusMTManagement.UserMaster
            ) AS K
            WHERE K.RNK = 1 
            -- and UserName='ZYMT-131'
        ) AS M ON A.UserName = M.UserName
        WHERE MONTH(TRY_CONVERT(date, A.[Date], 105)) = 1
          AND A.Month = 1 AND Year =2026 AND LOWER(A.FullName) NOT LIKE '%test%'
          --AND (A.UserName LIKE '%ISP%' OR A.UserName LIKE '%MR%') 
          AND LOWER(A.UserName) NOT LIKE '%test%'
    )
    SELECT distinct Designation
           --RANK() OVER (PARTITION BY UserName ORDER BY Date2 DESC) AS CURR_DAYFLAG
    FROM Attendance_CTE;
    
    